# Pearson's chi-squared test from scratch with Python:


In [1]:
# import libraries
import numpy as np 
import pandas as pd
import scipy
from scipy.stats import chi2

In [2]:
# create contingency table with floats to avoid datatype issues with pd.DataFrame.at 
ar=np.array([[12.0, 12.0],[11.0, 32.0]])    
df=pd.DataFrame(ar, columns=["Disease", "No Disease"])
df.index=["Exposed", "Unexposed"] 
df 

,Disease,No Disease
Exposed,12.0,12.0
Unexposed,11.0,32.0


In [3]:
df2=df.copy() # create contingency table with the marginal totals and the grand total. 
df2.loc['Column_Total']= df2.sum(numeric_only=True, axis=0)
df2.loc[:,'Row_Total'] = df2.sum(numeric_only=True, axis=1)
df2

,Disease,No Disease,Row_Total
Exposed,12.0,12.0,24.0
Unexposed,11.0,32.0,43.0
Column_Total,23.0,44.0,67.0


In [4]:
n=df2.at["Column_Total", "Row_Total"]  # grand total 

exp=df2.copy()               # create dataframe with expected counts
for x in exp.index[0:-1]:
    for y in exp.columns[0:-1]:
        # round expected values to 6 decimal places to get the maximum available precision:
        v= (((df2.at[x, "Row_Total"]) * (df2.at["Column_Total", y])   )   /n ).round(6) 
        exp.at[x,y]=float(v)

exp = exp.iloc[[0, 1], [0, 1]]
exp

,Disease,No Disease
Exposed,8.238806,15.761194
Unexposed,14.761194,28.238806


In [5]:
tstat = np.sum(((df-exp)**2/exp).values) # calculate chi-squared test statistic
tstat

4.0739497437848415

In [6]:
dof = (len(df.columns)-1)*(len(df.index)-1) # determine degrees of freedom 
dof

1

In [7]:
pval=1-chi2.cdf(tstat, dof) # subtract the cumulative distribution function from 1
pval

0.043549331095157795

In [8]:
from scipy.stats import chi2_contingency # import Scipy's built-in function

tstat_scipy,pval_scipy,ddof_scipy,exp_scipy=chi2_contingency(df, correction=False) # "correction=False" means no Yates' correction is used! 
print("Chi-squared test statistic without Yates correction (Scipy): " + str(tstat_scipy))
print("P-value without Yates correction (Scipy): " + str(pval_scipy))

Chi-squared test statistic without Yates correction (Scipy): 4.073949811563561
P-value without Yates correction (Scipy): 0.043549329347933916


# Chi-squared test with Yates correction:
All the aforementioned steps are basically the same but we use the following (adjusted) formula to determine our test statistic:

$$ \chi_{Yates}^{2}=  \sum_{i} \frac{  \big(  \big|O_i-E_i\big|-0.5  \big)^{2} }{E_i}$$


In [9]:
df 

,Disease,No Disease
Exposed,12.0,12.0
Unexposed,11.0,32.0


In [10]:
exp

,Disease,No Disease
Exposed,8.238806,15.761194
Unexposed,14.761194,28.238806


In [11]:
dof = (len(df.columns)-1)*(len(df.index)-1)
dof

1

In [12]:
# Apply Yates' correction by subtracting 0.5 from the absolute difference between observed and expected counts: 
tstat_yates= np.sum((((np.abs(df-exp)-0.5)**2)  / (exp)).values)
print("Chi-squared test statistic with Yates correction: " + str(tstat_yates))

pval=1-   chi2.cdf(tstat_yates, dof)
print("P-value with Yates correction: " + str(pval))

Chi-squared test statistic with Yates correction: 3.0627917403923197
P-value with Yates correction: 0.08010393193120602


In [13]:
from scipy.stats import chi2_contingency
tstat_scipy,pval_scipy,ddof_scipy,exp_scipy=chi2_contingency(df, correction=True)# "correction=True" to apply Yates' correction
print("Chi-squared test statistic with Yates correction (Scipy): " + str(tstat_scipy))
print("P-value with Yates correction (Scipy): " + str(pval_scipy))

Chi-squared test statistic with Yates correction (Scipy): 3.062791798801972
P-value with Yates correction (Scipy): 0.08010392905209013


### (Fisher's exact test)
Out of curiosity, Fisher's exact test would give us the following p-value: 

In [14]:
import scipy.stats as stats

oddsratio, pvalue_fisher = stats.fisher_exact(df)   
pvalue_fisher

0.061104061670370635